# Deploying a Model on Paperspace

In this notebook, an image classification model will be created to classify hand-written numbers, like the mnist dataset. This model will then be deployed using Paperspace.

It all follows from these blog posts: 

https://blog.paperspace.com/deploying-deep-learning-models-flask-web-python/
https://blog.paperspace.com/deploying-deep-learning-models-part-ii-hosting-on-paperspace/

## Import Modules

In [2]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras import backend as K

## Define Hyperparameters

In [3]:
num_classes = 10
batch_size = 128
epochs=12

## Load Images

In [21]:
# resolution
img_rows, img_cols = 28, 28

# load data
(x_train, y_train), (x_test, y_test) = mnist.load_data()

## Preprocess Data

In [22]:
if K.image_data_format() == "channels_first":
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

In [23]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255
x_test /= 255

print("x_train shape: ", x_train.shape)
print("x_test shape: ", x_test.shape)

x_train shape:  (60000, 28, 28, 1)
x_test shape:  (10000, 28, 28, 1)


In [24]:
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

## Define Model Architecture

In [25]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, kernel_size=(3,3), activation="relu", 
                          input_shape=input_shape),
    tf.keras.layers.Conv2D(64, kernel_size=(3,3), activation="relu"),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer = tf.keras.optimizers.Adadelta(),
             loss = tf.keras.losses.categorical_crossentropy,
             metrics=['accuracy'])

## Training

In [26]:
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs,verbose=1,
          validation_data=(x_test,y_test))

Epoch 1/12
469/469 [==============================] - 41s 88ms/step - loss: 2.3006 - accuracy: 0.1041 - val_loss: 2.2489 - val_accuracy: 0.2231
Epoch 2/12
469/469 [==============================] - 41s 87ms/step - loss: 2.2419 - accuracy: 0.1943 - val_loss: 2.1728 - val_accuracy: 0.4072
Epoch 3/12
469/469 [==============================] - 40s 86ms/step - loss: 2.1677 - accuracy: 0.3113 - val_loss: 2.0691 - val_accuracy: 0.6146
Epoch 4/12
469/469 [==============================] - 40s 86ms/step - loss: 2.0647 - accuracy: 0.4223 - val_loss: 1.9278 - val_accuracy: 0.6923
Epoch 5/12
469/469 [==============================] - 40s 86ms/step - loss: 1.9340 - accuracy: 0.5029 - val_loss: 1.7473 - val_accuracy: 0.7384
Epoch 6/12
469/469 [==============================] - 42s 89ms/step - loss: 1.7651 - accuracy: 0.5641 - val_loss: 1.5366 - val_accuracy: 0.7639
Epoch 7/12
469/469 [==============================] - 41s 86ms/step - loss: 1.5780 - accuracy: 0.6046 - val_loss: 1.3218 - val_accuracy:

## Evaluate the Model

In [28]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Test Loss: ", score[0])
print("Test Accuracy: ", score[1])

Test Loss:  0.687073826789856
Test Accuracy:  0.8474000096321106


## Save the Model

In [29]:
# save model architecture
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

# save model weights
model.save_weights("model.h5")

## Load model

In [33]:
# load architecture
json_file = open("model.json", "r")
loaded_model_json = json_file.read()
json_file.close()

# get model from json_file
loaded_model = tf.keras.models.model_from_json(loaded_model_json)

# load weights
loaded_model.load_weights("model.h5")
print("Loaded Model from Disk")

loaded_model.compile(loss='categorical_crossentropy',
                      optimizer="adam",
                      metrics=['accuracy'])

Loaded Model from Disk


## Create Flask App to Serve the Model

The Flask app will be called `app.py` and is a separate file. The code that appears in that file is below. No need to run the following cell.

In [1]:
from flask import Flask, render_template, request
from skimage.io import imsave, imread
from skimage.transform import resize
import numpy as numpy
import tensorflow.keras.models
import re
import sys
import os
import base64
sys.path.append(os.path.abspath("./model"))
from load import *

global graph, model

model, graph = init()

app = Flask(__name__)

@app.route('/')
def index_view():
	return render_template('index.html')

def convert_image(imgData1):
	imgstr = re.search(b'base64,(.*)', imgdata1).group(1)
	with open('output.png', 'wb') as output:
		output.write(base64.b64decode(imgstr))

@app.route('/predict/', methods=['GET', 'POST'])
def predict():
	imgDate - request.get_data()
	convertImage(imgData)
	x = imread('output.png', mode='L')
	x = np.invert(x)
	x = resize(x, (28,28))
	x = x.reshape(1, 28, 28, 1)

	with graph.as_default():
		out = model.predict(x)
		print(out)
		print(np.argmax(out, axis=1))

		response = np.array_str(np.argmax(out, axis=1))
		return response

if __name__ == '__main__':
	app.run(debug=True, port=8000)

Loaded Model from Disk
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:8000/ (Press CTRL+C to quit)
 * Restarting with stat


SystemExit: 1

/Users/johnrizcallah/opt/anaconda3/envs/tf/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3435: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
